<a href="https://colab.research.google.com/github/shortcipher3/nd131_smart_queuing/blob/master/notebooks/Create_Python_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step1: Create the Python Script

In the cell below, you will need to complete the Python script and run the cell to generate the file using the magic `%%writefile` command. Your main task is to complete the following methods for the `PersonDetect` class:
* `load_model`
* `predict`
* `draw_outputs`
* `preprocess_outputs`
* `preprocess_inputs`

For your reference, here are all the arguments used for the argument parser in the command line:
* `--model`:  The file path of the pre-trained IR model, which has been pre-processed using the model optimizer. There is automated support built in this argument to support both FP32 and FP16 models targeting different hardware.
* `--device`: The type of hardware you want to load the model on (CPU, GPU, MYRIAD, HETERO:FPGA,CPU)
* `--video`: The file path of the input video.
* `--output_path`: The location where the output stats and video file with inference needs to be stored (results/[device]).
* `--max_people`: The max number of people in queue before directing a person to another queue.
* `--threshold`: The probability threshold value for the person detection. Optional arg; default value is 0.60.

In [0]:
%%writefile person_detect.py

import numpy as np
import time
from openvino.inference_engine import IENetwork, IECore
import os
import cv2
import argparse
import sys
import logging


class Queue:
    '''
    Class for dealing with queues
    '''
    def __init__(self):
        self.queues=[]

    def add_queue(self, points):
        self.queues.append(points)

    def get_queues(self, image):
        for q in self.queues:
            x_min, y_min, x_max, y_max=q
            frame=image[y_min:y_max, x_min:x_max]
            yield frame
    
    def check_coords(self, coords):
        d={k+1:0 for k in range(len(self.queues))}
        for coord in coords:
            for i, q in enumerate(self.queues):
                if coord[0]>q[0] and coord[2]<q[2]:
                    d[i+1]+=1
        return d

    def check_detections(self, detections):
        """
        I chose to use a different datastructure, in my experience bounding
        box output varies framework to framework - eg tensorflow, object
        detection, yolo object detection, and some mxnet models I have used all
        have different bbox specifications, so I prefer to abstract away the
        indices when processing output.
        """
        d={k+1:0 for k in range(len(self.queues))}
        for detection in detections:
            for i, q in enumerate(self.queues):
                if detection['x_min']>q[0] and detection['x_max']<q[2]:
                    d[i+1]+=1
        return d

class PersonDetect:
    '''
    Class for the Person Detection Model.
    '''

    def __init__(self, model_name, device, threshold=0.60):
        self.model_weights=model_name+'.bin'
        self.model_structure=model_name+'.xml'
        self.device=device
        self.threshold=threshold

        try:
            self.model=IENetwork(self.model_structure, self.model_weights)
        except Exception as e:
            raise ValueError("Could not Initialise the network. Have you enterred the correct model path?")

        self.input_name=next(iter(self.model.inputs))
        self.input_shape=self.model.inputs[self.input_name].shape
        self.output_name=next(iter(self.model.outputs))
        self.output_shape=self.model.outputs[self.output_name].shape

    def load_model(self):
        '''
        Load the model for the given device
        '''
        core = IECore()
        self.net = core.load_network(network=model, device_name=args.device, num_requests=1)
        
    def predict(self, image):
        '''
        Given an image make predictions
        '''
        input_dict = self.preprocess_input(image)
        outputs = self.net.infer(input_dict)
        detections = self.preprocess_outputs(outputs)
        image = self.draw_outputs(detections, image)
        return detections
        return coords, image
    
    def draw_outputs(self, detections, image):
        '''
        draw the bounding boxes
        '''
        for det in detections:
            x_min = int(det['x_min'] * img.shape[1])
            x_max = int(det['x_max'] * img.shape[1])
            y_min = int(det['y_min'] * img.shape[0])
            y_max = int(det['y_max'] * img.shape[0])
            cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (125, 255, 51),
                          thickness=2)
            cv2.putText(img,
                        f'score: {det["conf"]:.2f} label: {det["label"]}',
                        (x_min, y_min),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        .5,
                        (0,0,255),
                        2,
                        cv2.LINE_AA)
        return img

    def preprocess_outputs(self, outputs):
        '''
        Convert the output
        '''
        logging.info(arr)
        arr = outputs[self.output_name]
        num_detections = arr.shape[0]
        detections = []
        for k in range(num_detections):
             x_min = arr[k, 3]
             x_max = arr[k, 5]
             y_min = arr[k, 4]
             y_max = arr[k, 6]
             width = np.abs(x_max - x_min)
             height = np.abs(y_max - y_min)
             area = width * height
             detections.append({'image_id': arr[k, 0],
                                'label': arr[k, 1],
                                'conf': arr[k, 2],
                                'x_min': x_min,
                                'y_min': y_min,
                                'x_max': x_max,
                                'y_max': y_max,
                                'width': width,
                                'height': height,
                                'area': area})
        logging.info(detections)
        return detections

    def preprocess_input(self, image):
        '''
        Convert the image to a batch
        '''
        batch = cv2.resize(image, (self.input_shape[2], self.input_shape[3]))
        batch = batch.transpose((2, 0, 1))
        input_dict = {
            self.input_name: batch[np.newaxis, :, :, :]
        }
        return input_dict


def main(args):
    model=args.model
    device=args.device
    video_file=args.video
    max_people=args.max_people
    threshold=args.threshold
    output_path=args.output_path

    start_model_load_time=time.time()
    pd = PersonDetect(model, device, threshold)
    pd.load_model()
    total_model_load_time = time.time() - start_model_load_time

    queue=Queue()
    
    try:
        queue_param=np.load(args.queue_param)
        for q in queue_param:
            queue.add_queue(q)
    except:
        print("error loading queue param file")

    try:
        cap = cv2.VideoCapture(video_file)
    except FileNotFoundError:
        print("Cannot locate video file: " + video_file)
    except Exception as e:
        print("Something else went wrong with the video file: ", e)
    
    initial_w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    initial_h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    video_len = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    out_video = cv2.VideoWriter(os.path.join(output_path, 'output_video.mp4'), cv2.VideoWriter_fourcc(*'avc1'), fps, (initial_w, initial_h), True)
    
    counter=0
    start_inference_time=time.time()

    try:
        while cap.isOpened():
            ret, frame=cap.read()
            if not ret:
                break
            counter+=1
            
            coords, image = pd.predict(frame)
            num_people = queue.check_detections(detections)
            print(f"Total People in frame = {len(coords)}")
            print(f"Number of people in queue = {num_people}")
            out_text=""
            y_pixel=25
            
            for k, v in num_people.items():
                out_text += f"No. of People in Queue {k} is {v} "
                if v >= int(max_people):
                    out_text += f" Queue full; Please move to next Queue "
                cv2.putText(image, out_text, (15, y_pixel), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
                out_text=""
                y_pixel+=40
            out_video.write(image)
            
        total_time=time.time()-start_inference_time
        total_inference_time=round(total_time, 1)
        fps=counter/total_inference_time

        with open(os.path.join(output_path, 'stats.txt'), 'w') as f:
            f.write(str(total_inference_time)+'\n')
            f.write(str(fps)+'\n')
            f.write(str(total_model_load_time)+'\n')

        cap.release()
        cv2.destroyAllWindows()
    except Exception as e:
        print("Could not run Inference: ", e)

if __name__=='__main__':
    parser=argparse.ArgumentParser()
    parser.add_argument('--model', required=True)
    parser.add_argument('--device', default='CPU')
    parser.add_argument('--video', default=None)
    parser.add_argument('--queue_param', default=None)
    parser.add_argument('--output_path', default='/results')
    parser.add_argument('--max_people', default=2)
    parser.add_argument('--threshold', default=0.60)
    
    args=parser.parse_args()

    logging.basicConfig(level=logging.INFO)
    main(args)

# Next Step

Now that you've run the above cell and created your Python script, you will create your job submission shell script in the next workspace.

**Note**: As a reminder, if you need to make any changes to the Python script, you can come back to this workspace to edit and run the above cell to overwrite the file with your changes.